In [1]:
import ipynb
from ipynb.fs.full.A_libs_26sep import *
from ipynb.fs.full.B_funs_26sep import *

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

### Configuration

In [2]:
output_dir = create_output_dir()
seed_everything(seed=42)

Directory  ../results/_FINAL_ANN_CPS_RANDOM_KMEANS7_SMOOTH_NO  Created 


In [3]:
class CFG:
    num_folds = 10
    use_var_enc = True
    variance_thres = 0.60
    quantile_transform = True
    use_rankgauss = True
    
    selec_top = True
    original_feats = True #False remove also top features
    
    use_pca = True
    pca_comp_genes = 29 #30
    pca_comp_cells = 4 #18
    
    use_fastica = False
    fastica_comp_genes = 10
    fastica_comp_cells = 5
    
    use_kridge = True
    use_kridge_group = False
    
    use_xgb = False
    use_bayes = False


### DataBases

In [4]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/train_drug.csv')

drug_dict = dict(zip(train_drug['sig_id'],train_drug['drug_id']))

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

train_orig = train_features.merge(train_targets_scored, on='sig_id', how='left')
# train_orig = train_orig.merge(train_targets_nonscored, on='sig_id', how='left')
train_orig = train_orig.merge(train_drug, on='sig_id', how='left')

train_noctl = train_orig[train_orig['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
train_ctl = train_orig[train_orig['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

test_noctl = test_features[test_features['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
test_ctl = test_features[test_features['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

target = train_noctl[train_targets_scored.columns]

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
target_cols = train_targets_scored.drop('sig_id', axis=1).columns.values.tolist()
# target_nonscored_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()
other_feats = []


# mskf = MultilabelStratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=43)
# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_noctl, y=target)):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)
# sum_ones = train_noctl[target_cols].sum(axis=0)

folds = train_noctl.copy()
print(train_orig.shape)
print(target.shape)
print(sample_submission.shape)

(23814, 1083)
(21948, 207)
(3982, 207)


### Function Calc Folds

In [5]:
def calc_folds(seed_fold):
    seed_everything(seed=seed_fold)
    
    folds = train_noctl.copy()
    mskf = MultilabelStratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=seed_fold)
    train_noctl['kfold'] = 0.0
    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_noctl, y=target)):
        train_noctl.loc[v_idx, 'kfold'] = int(f)
    train_noctl['kfold']  = train_noctl['kfold'].astype('int')
    return train_noctl['kfold'].values

    
#     seed_everything(seed=seed_fold)
#     # LOCATE DRUGS
#     vc = train_noctl.drug_id.value_counts()
#     vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
#     vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()

# #     vc1 = vc.loc[vc <= 18].index.sort_values()
# #     vc2 = vc.loc[vc > 18].index.sort_values()

#      # STRATIFY DRUGS 18X OR LESS
#     dct1 = {}; dct2 = {}
#     skf = MultilabelStratifiedKFold(n_splits = CFG.num_folds, shuffle = True, random_state = seed_fold)
#     # skf = KFold(n_splits = CFG.num_folds, shuffle = True, random_state = 42)
#     tmp = train_noctl.groupby('drug_id')[target_cols].mean().loc[vc1]

#     for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
#         dd = {k:fold for k in tmp.index[idxV].values}
#         dct1.update(dd)

#     # STRATIFY DRUGS MORE THAN 18X
#     skf = MultilabelStratifiedKFold(n_splits = CFG.num_folds, shuffle = True, random_state = seed_fold)
#     # skf = KFold(n_splits = CFG.num_folds, shuffle = True, random_state = 42)
#     tmp = train_noctl.loc[train_noctl['drug_id'].isin(vc2)].reset_index(drop = True)
#     for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
#         dd = {k:fold for k in tmp.sig_id[idxV].values}
#         dct2.update(dd)
    
#     # ASSIGN FOLDS
#     train_noctl['kfold'] = train_noctl.drug_id.map(dct1)
#     train_noctl.loc[train_noctl['kfold'].isna(),'kfold'] = train_noctl.loc[train_noctl['kfold'].isna(),'sig_id'].map(dct2)
#     train_noctl['kfold']  = train_noctl['kfold'].astype('int')
#     return train_noctl['kfold'].values     

### RankGauss

In [6]:
#From: https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn
folds_cp = folds.copy()
test_noctl_cp = test_noctl.copy()

if CFG.use_rankgauss:
    for col in (GENES + CELLS):
        transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
        vec_len = len(folds_cp[col].values)
        vec_len_test = len(test_noctl_cp[col].values)
        raw_vec = folds_cp[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)

        folds_cp[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
        test_noctl_cp[col] = transformer.transform(test_noctl_cp[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### KMeans Features

In [7]:
from sklearn.cluster import KMeans
import gc

def create_cluster(train, test, features, kind = 'g', n_clusters = 35):
    train_ = train[features].copy()
    test_ = test[features].copy()
    
    scaler = StandardScaler()
    train_ = scaler.fit_transform(train_)
    test_ = scaler.transform(test_)
    model = KMeans(n_clusters = n_clusters, random_state = 123).fit(train_)
    name_cols = [f'kmeans_{kind}_{i}' for i in range(n_clusters)]
    train = pd.concat([train, pd.DataFrame(model.transform(train_), columns=name_cols)], axis=1)
    test = pd.concat([test, pd.DataFrame(model.transform(test_), columns=name_cols)], axis=1)
    print(train.shape, test.shape)
    return train, test
    
folds_cp2 = folds.copy()
test_noctl_cp2 = test_noctl.copy()
folds_cp2, test_noctl_cp2 = create_cluster(folds_cp2,test_noctl_cp2, GENES, kind = 'g', n_clusters = 7)
folds_cp2, test_noctl_cp2 = create_cluster(folds_cp2,test_noctl_cp2, CELLS, kind = 'c', n_clusters = 7)

folds_3models = folds_cp2[list(folds_cp2.columns.values[np.where(folds_cp2.columns=='kmeans_g_0')[0][0]:])]
test_noctl_3model = test_noctl_cp2[list(folds_cp2.columns.values[np.where(folds_cp2.columns=='kmeans_g_0')[0][0]:])]
del folds_cp2, test_noctl_cp2
gc.collect()

# train_features ,test_features=fe_cluster(train_features,test_features)

(21948, 1089) (3624, 882)
(21948, 1096) (3624, 889)


0

### PCA Features

In [8]:
if CFG.use_pca:
    seed_everything(seed=42)
    etiq = ['G','C']
    num_pca = [CFG.pca_comp_genes, CFG.pca_comp_cells]
    for niter, cols in enumerate([GENES, CELLS]):
        # PCA for train with folds
        num_comp = num_pca[niter]
        columns_pca = [f'pca_{etiq[niter]}-{i}' for i in range(num_comp)]
        other_feats += columns_pca

        # PCA for train
        pca = PCA(n_components=num_comp, random_state=42).fit(folds_cp[cols])
        train_pca = pca.transform(folds_cp[cols])
        train_pca = pd.DataFrame(train_pca, columns=columns_pca)
        folds = pd.concat((folds, train_pca), axis=1)
        
        # PCA for train
        test_pca = pca.transform(test_noctl_cp[cols])
        test_pca = pd.DataFrame(test_pca, columns=columns_pca)
        test_noctl = pd.concat((test_noctl, test_pca), axis=1)
        print(folds.shape, test_noctl.shape)

(21948, 1111) (3624, 904)
(21948, 1115) (3624, 908)


### Select Top Features

In [9]:
# from sklearn.utils import check_random_state  # type: ignore

# ### from eli5
# def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False, random_state=None):
#     rng = check_random_state(random_state)

#     if columns_to_shuffle is None:
#         columns_to_shuffle = range(X.shape[1])

#     if pre_shuffle:
#         X_shuffled = X.copy()
#         rng.shuffle(X_shuffled)

#     X_res = X.copy()
#     for columns in tqdm(columns_to_shuffle):
#         if pre_shuffle:
#             X_res[:, columns] = X_shuffled[:, columns]
#         else:
#             rng.shuffle(X_res[:, columns])
#         yield X_res
#         X_res[:, columns] = X[:, columns]

# def get_score_importances(
#         score_func,  # type: Callable[[Any, Any], float]
#         X,
#         y,
#         n_iter=5,  # type: int
#         columns_to_shuffle=None,
#         random_state=None
#     ):
#     rng = check_random_state(random_state)
#     base_score = score_func(X, y)
#     scores_decreases = []
#     for i in range(n_iter):
#         scores_shuffled = _get_scores_shufled(
#             score_func, X, y, columns_to_shuffle=columns_to_shuffle,
#             random_state=rng, base_score=base_score
#         )
#         scores_decreases.append(scores_shuffled)

#     return base_score, scores_decreases

# def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None,
#                         random_state=None):
#     Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
#     res = []
#     for X_shuffled in Xs:
#         res.append(-score_func(X_shuffled, y) + base_score)
#     return res

# def metric(y_true, y_pred):
#     metrics = []
#     for i in range(y_pred.shape[1]):
#         if y_true[:, i].sum() > 1:
#             metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float)))
#     return np.mean(metrics)   

# perm_imp = np.zeros(train.shape[1])
# all_res = []
# for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(train_targets)):
#     print(f'Fold {n}')

#     model = create_model(len(train.columns))
#     checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
#     reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
#     cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
#                                      save_weights_only = True, mode = 'min')
#     model.fit(train.values[tr],
#                   train_targets.values[tr],
#                   validation_data=(train.values[te], train_targets.values[te]),
#                   epochs=35, batch_size=128,
#                   callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
#                  )
        
#     model.load_weights(checkpoint_path)
        
#     def _score(X, y):
#         pred = model.predict(X)
#         return metric(y, pred)

#     base_score, local_imp = get_score_importances(_score, train.values[te], train_targets.values[te], n_iter=1, random_state=0)
#     all_res.append(local_imp)
#     perm_imp += np.mean(local_imp, axis=0)
#     print('')
    
# top_feats = np.argwhere(perm_imp < 0).flatten()
# top_feats


In [10]:
if CFG.selec_top:
    seed_everything(seed=42)
    # https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2/data
    top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
            18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
            32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
            47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
            61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
            74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
            89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
           102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
           115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
           129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
           144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
           158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
           171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
           184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
           198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
           213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
           227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
           240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
           254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
           267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
           281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
           295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
           310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
           324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
           337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
           350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
           363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
           378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
           392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
           405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
           419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
           432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
           447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
           463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
           476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
           490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
           506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
           522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
           538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
           552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
           571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
           586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
           600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
           618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
           631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
           645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
           660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
           673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
           686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
           701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
           718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
           733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
           748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
           762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
           775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
           789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
           804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
           821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
           837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
           854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
           870, 871, 872, 873, 874]
    # print(len(top_feats))
    selected_features = np.array(['cp_type','cp_time', 'cp_dose']+GENES+CELLS)[top_feats].tolist()
    GENES = [col for col in selected_features if 'g-' in col]
    CELLS = [col for col in selected_features if 'c-' in col]
    numeric_cols = other_feats + GENES + CELLS
    folds = folds[['sig_id','cp_time','cp_dose']+numeric_cols+target_cols]
    test_noctl = test_noctl[['sig_id','cp_time','cp_dose']+numeric_cols]
    print(folds.shape, test_noctl.shape)

(21948, 1025) (3624, 819)


### Reduce Dataset using Variance

In [11]:
class VarianceThreshold:
    def __init__(self, threshold):
        self.threshold = threshold
    def fit(self, df, cont_cols):
        self.cont_cols = cont_cols
        self.var = folds[cont_cols].var()
        good_cols = self.var[self.var > self.threshold]
        self.index = good_cols.index.to_list()
        self.dropcols = [x for x in cont_cols if x not in self.var[self.var > self.threshold].index.to_list()]
        self.validcols = [x for x in cont_cols if x in self.var[self.var > self.threshold].index.to_list()]
    def transform(self, df):
        return df.drop(self.dropcols, axis=1)
    def fit_transform(self, df, cont_cols):
        self.fit(df, cont_cols)
        return self.transform(df), self.validcols

In [12]:
if CFG.use_var_enc:
    cont_cols_ini = [i for i in test_noctl.columns if i not in ['sig_id', 'cp_time', 'cp_dose']]
    print('Variance Threshold:', CFG.variance_thres)
    print(folds.shape, test_noctl.shape)
    VarThres = VarianceThreshold(CFG.variance_thres)
    folds, cont_cols = VarThres.fit_transform(folds, cont_cols_ini)
    test_noctl = VarThres.transform(test_noctl)
    print(folds.shape, test_noctl.shape)


Variance Threshold: 0.6
(21948, 1025) (3624, 819)
(21948, 1011) (3624, 805)


### Include kridge features  removing with low num of ones

In [13]:
# df_03octe30 = pd.read_csv('../input/res_kridge_03oct_e30_v2.csv')
# df_03octe31 = pd.read_csv('../input/res_kridge_03oct_e31_v2.csv')
# df_03oct = pd.concat([df_03octe30, df_03octe31], sort=False)
# df_03octe30.shape, df_03octe31.shape, df_03oct.shape
# df_03oct[df_03oct.sumones>10].AUC.mean()

# df_16oct = pd.read_csv('../results/_29_Create_KRIDGE_feats_16oct_e30/res.csv')
# df_16oct.head()
# df_16oct[df_16oct.sumones>10].AUC.mean()
# res_dos = []
# for max_ones in range(40):
#     res_dos.append(dict(max_ones=max_ones, oct3=df_03oct[df_03oct.sumones>max_ones].AUC.mean(), oct16=df_16oct[df_16oct.sumones>max_ones].AUC.mean()))
# res_dos = pd.DataFrame(res_dos)

# sns.lineplot(data=res_dos, x='max_ones', y='oct3',label='oct3')
# sns.lineplot(data=res_dos, x='max_ones', y='oct16',label='oct16')

In [14]:
if CFG.use_kridge:
#     seed_everything(seed=42)
#     folds_feat = pd.read_csv('../input/folds_feats.csv')
#     test_feat = pd.read_csv('../input/test_feat.csv')
#     kridge_cols = ['sig_id']+[col for col in folds_feat if 'kridge' in col]
#     folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
#     test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
#     print(folds.shape, test_noctl.shape)
    
    min_num_ones = 20
    folds_feat = pd.read_csv('../results/_29_Create_KRIDGE_feats_16oct_e30/folds.csv')
    test_feat = pd.read_csv('../results/_29_Create_KRIDGE_feats_16oct_e30/test.csv')
    
    colsk = [col.replace('kridge_','') for col in folds_feat if 'kridge' in col]
    colsk_selec = []
    target_selec = []
    for colk in colsk:
        if np.sum(folds[colk].sum())>=min_num_ones:
            colsk_selec += ['kridge_'+colk]
            target_selec += [colk]
    kridge_cols = ['sig_id']+colsk_selec

    folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
    print(len(colsk), len(colsk_selec), folds.shape, test_noctl.shape)
    
    kridge_cols = [col for col in folds if 'kridge_' in col]
#     print('logloss=',log_loss_multi(folds[target_selec].values, folds[kridge_cols].values))    

206 153 (21948, 1164) (3624, 958)


In [15]:
if CFG.use_kridge_group:

    colsk_selec = ['kridge_group003', 'kridge_group007', 'kridge_group008', 'kridge_group009', 'kridge_group010', 'kridge_group012', 'kridge_group016', 'kridge_group020', 
                    'kridge_group025', 'kridge_group028', 'kridge_group029', 'kridge_group035', 'kridge_group036', 'kridge_group039', 'kridge_group049', 'kridge_group051', 
                    'kridge_group053', 'kridge_group056', 'kridge_group057', 'kridge_group063', 'kridge_group066', 'kridge_group068', 'kridge_group069', 'kridge_group075', 
                    'kridge_group076', 'kridge_group080', 'kridge_group081', 'kridge_group085', 'kridge_group088', 'kridge_group091', 'kridge_group093', 'kridge_group094', 
                    'kridge_group095', 'kridge_group098', 'kridge_group099', 'kridge_group100', 'kridge_group101', 'kridge_group103', 'kridge_group105', 'kridge_group106', 
                    'kridge_group107', 'kridge_group109', 'kridge_group115', 'kridge_group118', 'kridge_group120', 'kridge_group128', 'kridge_group129', 'kridge_group130', 
                    'kridge_group131', 'kridge_group134', 'kridge_group137', 'kridge_group138', 'kridge_group139', 'kridge_group141', 'kridge_group142', 'kridge_group143', 
                    'kridge_group144', 'kridge_group150', 'kridge_group152', 'kridge_group153', 'kridge_group154', 'kridge_group155', 'kridge_group156', 'kridge_group158', 
                    'kridge_group159', 'kridge_group162', 'kridge_group163', 'kridge_group165', 'kridge_group168', 'kridge_group173', 'kridge_group174', 'kridge_group175', 
                    'kridge_group176', 'kridge_group177', 'kridge_group178', 'kridge_group182', 'kridge_group184', 'kridge_group186', 'kridge_group187', 'kridge_group191', 
                    'kridge_group192', 'kridge_group193', 'kridge_group198', 'kridge_group199', 'kridge_group201', 'kridge_group202', 'kridge_group206', 'kridge_group208', 
                    'kridge_group209', 'kridge_group211', 'kridge_group213', 'kridge_group215', 'kridge_group216', 'kridge_group219', 'kridge_group220', 'kridge_group221', 
                    'kridge_group224', 'kridge_group225', 'kridge_group227', 'kridge_group229', 'kridge_group233', 'kridge_group234', 'kridge_group235', 'kridge_group236', 
                    'kridge_group238', 'kridge_group240', 'kridge_group243', 'kridge_group244', 'kridge_group246', 'kridge_group247', 'kridge_group249', 'kridge_group255', 
                    'kridge_group256', 'kridge_group257', 'kridge_group262', 'kridge_group263', 'kridge_group265', 'kridge_group266', 'kridge_group268', 'kridge_group269', 
                    'kridge_group271', 'kridge_group272', 'kridge_group273', 'kridge_group274', 'kridge_group276', 'kridge_group278', 'kridge_group284', 'kridge_group286', 
                    'kridge_group287', 'kridge_group288', 'kridge_group289', 'kridge_group291', 'kridge_group292', 'kridge_group293', 'kridge_group294', 'kridge_group298', 
                    'kridge_group301', 'kridge_group304', 'kridge_group305', 'kridge_group306', 'kridge_group307', 'kridge_group308', 'kridge_group311', 'kridge_group314', 
                    'kridge_group316', 'kridge_group317', 'kridge_group320', 'kridge_group323', 'kridge_group324', 'kridge_group325']
    
    folds_feat = pd.read_csv('../results/_2004_Create_KRIDGE_groupedbytargets/folds.csv')
    test_feat = pd.read_csv('../results/_2004_Create_KRIDGE_groupedbytargets/test.csv')
    
    kridge_cols = ['sig_id']+colsk_selec
    folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
    print(len(colsk), len(colsk_selec), folds.shape, test_noctl.shape)

    kridge_cols = [col for col in folds if 'kridge_' in col]
    print(len(kridge_cols))

### Include XGB features removing with low num of ones

In [16]:
if CFG.use_xgb:
#     seed_everything(seed=42)
#     folds_feat = pd.read_csv('../input/folds_feats.csv')
#     test_feat = pd.read_csv('../input/test_feat.csv')
#     kridge_cols = ['sig_id']+[col for col in folds_feat if 'kridge' in col]
#     folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
#     test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
#     print(folds.shape, test_noctl.shape)
    min_num_ones = 25

    dir_model = '_31_model_XGB_withkridge_5oct_v2'
    dir_res = '../results/' + dir_model + '/'
    name_file = [i for i in os.listdir(dir_res) if 'dataframe' in i][0]
    with open(dir_res + name_file, 'rb') as file:
        folds_feat = pickle.load(file)
        test_feat = pickle.load(file)
        xbg_y_true = pickle.load(file)
        xgb_y_pred = pickle.load(file)
        xgb_feature_cols = pickle.load(file)
        xgb_target_cols = pickle.load(file)
    
    cols_xgb = xgb_target_cols
    cols_xgb_selec = []
    for colx in cols_xgb:
        if np.sum(folds[colx].sum())>=min_num_ones:
            cols_xgb_selec += [colx]
    print(len(cols_xgb_selec))
    
    folds_feat_selec = folds_feat[['sig_id'] + cols_xgb_selec]
    test_noctl_selec = test_feat[['sig_id'] + cols_xgb_selec]
    new_colnames  = ['xgb_'+i for i in cols_xgb_selec]
    folds_feat_selec.columns = ['sig_id'] +new_colnames
    test_noctl_selec.columns = ['sig_id'] +new_colnames

    folds = pd.merge(folds, folds_feat_selec[['sig_id'] + new_colnames], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_noctl_selec[['sig_id'] + new_colnames], on='sig_id')
    print(folds.shape, test_noctl.shape)

### Include features bayes

In [17]:
if CFG.use_bayes:
#     seed_everything(seed=42)
#     folds_feat = pd.read_csv('../input/folds_feats.csv')
#     test_feat = pd.read_csv('../input/test_feat.csv')
#     kridge_cols = ['sig_id']+[col for col in folds_feat if 'kridge' in col]
#     folds = pd.merge(folds, folds_feat[kridge_cols], on='sig_id')
#     test_noctl = pd.merge(test_noctl, test_feat[kridge_cols], on='sig_id')
#     print(folds.shape, test_noctl.shape)
    
    folds_feat = pd.read_csv('../input/bayes_folds_feats30sep_e30.csv')
    test_feat = pd.read_csv('../input/bayes_test_feats30sep_e30.csv')
    bayes_cols = ['sig_id']+[col for col in folds_feat if 'bayes' in col]
    folds = pd.merge(folds, folds_feat[bayes_cols], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_feat[bayes_cols], on='sig_id')
    
    folds_feat = pd.read_csv('../input/bayes_folds_feats30sep_e31.csv')
    test_feat = pd.read_csv('../input/bayes_test_feats30sep_e31.csv')
    bayes_cols = ['sig_id']+[col for col in folds_feat if 'bayes' in col]
    folds = pd.merge(folds, folds_feat[bayes_cols], on='sig_id')
    test_noctl = pd.merge(test_noctl, test_feat[bayes_cols], on='sig_id')

    print(folds.shape, test_noctl.shape)

### Quantile Transform of Features


In [18]:
if CFG.quantile_transform:
    qt = QuantileTransformer(output_distribution='normal')
    folds[cont_cols] = qt.fit_transform(folds[cont_cols])
    test_noctl[cont_cols] = qt.transform(test_noctl[cont_cols])

### Low or High 0.05 and 0.95 percentiles

In [19]:
# var_thresh = VarianceThreshold(threshold=0.7)
# data = train_features.append(test_features)
# data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

# train_features_transformed = data_transformed[ : train_features.shape[0]]
# test_features_transformed = data_transformed[-test_features.shape[0] : ]


# train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
#                               columns=['sig_id','cp_type','cp_time','cp_dose'])

# train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


# test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
#                              columns=['sig_id','cp_type','cp_time','cp_dose'])

# test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

# train_features

### Stats1

In [20]:
# train_noctl_inicial = train_orig[train_orig['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
# test_noctl_inicial = test_features[test_features['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

# folds['menos10_genes'] = (train_noctl_inicial[GENES]== -10.0).mean(axis=1)
# folds['menos10_cells'] = (train_noctl_inicial[CELLS]== -10.0).mean(axis=1)

# test_noctl['menos10_genes'] = (test_noctl_inicial[GENES]== -10.0).mean(axis=1)
# test_noctl['menos10_cells'] = (test_noctl_inicial[CELLS]== -10.0).mean(axis=1)
# print(folds.shape, test_noctl.shape)

In [21]:
folds = pd.concat((folds, folds_3models), axis=1)
test_noctl = pd.concat((test_noctl, test_noctl_3model), axis=1)
print(folds.shape, test_noctl.shape)
del folds_3models, test_noctl_3model
gc.collect()

(21948, 1178) (3624, 972)


294

### Preprocessing Steps

In [22]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [23]:
folds = process_data(folds)
test_noctl = process_data(test_noctl)

feature_cols = [c for c in folds.columns if c not in target_cols]
if not CFG.original_feats: feature_cols = [c for c in feature_cols if c not in GENES+CELLS]
feature_cols_ini = [c for c in feature_cols if c not in ['kfold','sig_id']] #, 'cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2']] #,'cp_dose','cp_time']]
len(feature_cols_ini)

974

In [24]:
# Standarize
for colname in feature_cols_ini:
    valor_fold = folds[colname].values
    valor_tst = test_noctl[colname].values
    mean_v = np.mean(valor_fold)
    std_v = np.std(valor_fold)
    if std_v==0:
        std_v=1e-5
    folds[colname] = (valor_fold-mean_v)/std_v
    test_noctl[colname] = (valor_tst-mean_v)/std_v
    print(colname, mean_v, std_v)

pca_G-0 -3.165934989350809e-05 1.000022942477717
pca_G-1 2.3190635233523497e-05 0.999589967313473
pca_G-2 -5.6528317499562846e-05 0.9998962469475889
pca_G-3 0.00014903320099432797 1.0001572798115808
pca_G-4 3.974275730476318e-05 0.9997665499591254
pca_G-5 7.361036404475456e-05 0.9997785695567266
pca_G-6 -3.722530770049402e-05 0.9997154816867923
pca_G-7 -4.176801412068981e-05 1.000241038168113
pca_G-8 -3.368941635777072e-05 0.9998038447404773
pca_G-9 5.6154809742764314e-05 0.9997265629134641
pca_G-10 -9.040019011965188e-05 0.9996772931738684
pca_G-11 -6.0787953075223735e-05 0.9997559892025474
pca_G-12 -5.8613588758092277e-05 0.9994258399589677
pca_G-13 -9.893185187019985e-05 0.9997849369040595
pca_G-14 -6.201536405677035e-05 0.9997168847993001
pca_G-15 -8.278464554952898e-05 0.9994982924307781
pca_G-16 -3.833382519051168e-06 1.0007720562770315
pca_G-17 -1.2060393554331708e-05 0.9995915837385395
pca_G-18 -6.393268281042678e-05 1.0000080139572305
pca_G-19 4.308745677184569e-05 0.999937528

g-192 0.00010064893766004415 0.9999783274804145
g-194 -0.00018718787945757798 1.0006105055574819
g-195 -0.013852815983566531 1.0532927617389494
g-196 -0.0006110470286010054 1.0015185249210554
g-197 0.0035337059096291912 1.013584532484019
g-199 -0.00012132593180523838 0.9999246631925712
g-200 -5.390627061139611e-05 0.999744241665109
g-201 0.010318955831430719 1.0400095166891867
g-202 -0.011705118187446958 1.0452692045182317
g-203 0.00015822317058976405 0.9998033941910466
g-205 -2.0336886546779832e-05 0.9993481599633784
g-206 -5.2236448396437755e-06 0.9993807051468245
g-207 -6.42168030457151e-05 0.9996756956232318
g-208 0.011885103957316704 1.0471444675514614
g-209 0.00014482179215395745 1.0002940892778733
g-210 0.0008783337807099783 1.002825444749206
g-211 6.391463749276528e-05 0.9996842489804277
g-212 9.224765209031916e-05 0.9996428538182857
g-213 -0.0002921543950907238 1.0069009975519547
g-214 -0.0001804455615557744 0.9997551990463769
g-215 0.003087975648939873 1.0129114210697607
g-21

g-413 -0.0001468403541331751 1.0001925170946624
g-414 -0.0003599127067282363 1.000930217463099
g-415 -0.0002686857295471402 1.0002494045867107
g-416 -6.791963316977216e-05 0.9999761015104424
g-417 0.04398780408857785 1.1545418812926918
g-418 3.542462677839846e-05 1.0004048143200053
g-419 0.000319205986986933 1.000742944935079
g-420 0.00010336036421055288 0.9996854542113559
g-421 -0.0007872990623085061 1.0025248490880707
g-422 0.00024766310526672927 1.0005347789924632
g-423 -0.003350512613591671 1.012815787059476
g-424 0.0029019921398072916 1.0112421366200424
g-425 -6.310806149709559e-05 0.999864405147701
g-426 -1.3444900450342463e-05 0.9997293936459648
g-427 0.0008912246550443706 1.00312686712729
g-428 1.263852251129537e-05 1.000515290135725
g-429 1.359700517569553e-06 1.0001169213933616
g-431 -0.00030565390329633933 1.0010724020477333
g-432 8.867503877749868e-05 0.9996213396026729
g-433 -0.0003008722062034469 1.001082736125666
g-434 -0.0006973488195214741 1.0025297806601539
g-436 0.00

g-662 7.015157967856963e-05 0.9994965392198006
g-663 0.0001688687480979832 0.9999207470122549
g-664 -0.00042005846076367554 1.0009943926043527
g-665 -0.0013798136765696932 1.0045421923782922
g-666 0.00010301547038907287 0.9998401190485882
g-667 -0.0002018860406917672 1.0000207916310362
g-668 3.4726471806727e-06 0.9996986042198724
g-669 -0.002129757378663416 1.0091844187531396
g-670 -0.00038038543436517806 1.0020252994861254
g-671 -0.00018948651900980288 1.0009080581091896
g-672 -0.009114142675257675 1.0355312533728362
g-673 -0.0003664295884627404 1.00084076747467
g-674 -0.004972941916024866 1.0195340499741892
g-675 6.8033420117136005e-06 0.999995698717916
g-676 0.0002420231334699168 1.0009428103694702
g-677 -0.000375217299066061 1.0012010115231371
g-678 -0.0007993598676973173 1.003684177859586
g-679 0.00010371155925009526 1.0001529978394759
g-680 6.258098411238148e-05 0.9998282006976745
g-681 -0.0002264154825830829 1.0007726968396988
g-682 0.00013113400828856762 0.9997174505029446
g-68

kridge_bromodomain_inhibitor 0.0020884272631574432 0.0026761989116421553
kridge_btk_inhibitor 0.0011912776390855784 0.00223079394290803
kridge_calcium_channel_blocker 0.012533839286073056 0.006325035174075205
kridge_cannabinoid_receptor_agonist 0.0018466728137510268 0.0022399646934310948
kridge_cannabinoid_receptor_antagonist 0.002438573469965061 0.0026470380656918617
kridge_carbonic_anhydrase_inhibitor 0.0015758098628883242 0.00189843446750408
kridge_casein_kinase_inhibitor 0.0015524950975176646 0.0020039714809702257
kridge_cc_chemokine_receptor_antagonist 0.004401901303348937 0.003404236892371685
kridge_cdk_inhibitor 0.00839555724725416 0.02557990007962004
kridge_chelating_agent 0.002359389588508494 0.0026121261716377797
kridge_chk_inhibitor 0.0006836890806613023 0.0016297091214179176
kridge_chloride_channel_blocker 0.0019300775900910868 0.00220065838761178
kridge_cholesterol_inhibitor 0.0021073894133025284 0.002260595075961982
kridge_cholinergic_receptor_antagonist 0.002297637938536

### Training Functions

In [25]:
from sklearn.metrics import roc_auc_score
def mean_auc(y_true, y_pred):
    auc_mean = []
    for ncol in range(y_true.shape[1]):
        if np.sum(y_true[:,ncol])>0:
            auc_mean.append(roc_auc_score(y_true[:,ncol], y_pred[:, ncol]))
    return np.mean(auc_mean)

In [26]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [27]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [28]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    y_true = []
    y_pred = []
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        if not scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step()
        
        final_loss += loss.item()
        y_true.append(targets.cpu().detach().numpy())
        y_pred.append(outputs.sigmoid().cpu().detach().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    
    control_loss = log_loss_multi(y_true, y_pred)
    final_loss /= len(dataloader)
    return final_loss, control_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    y_true = []
    y_pred = []
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        y_true.append(targets.cpu().detach().numpy())
        y_pred.append(outputs.sigmoid().cpu().detach().numpy())
        
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    control_loss = log_loss_multi(y_true, y_pred)
    final_loss /= len(dataloader)
    return final_loss, control_loss, y_true, y_pred

def inference_fn(model, dataloader, device):
    model.eval()
    y_pred = []
    for data in dataloader:
        inputs = data['x'].to(device)
        with torch.no_grad():
            outputs = model(inputs)
        y_pred.append(outputs.sigmoid().detach().cpu().numpy())
    y_pred = np.concatenate(y_pred)
    
    return y_pred

### Model

In [29]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(drop1_feat) #0.20
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(drop2_feat) #0.20
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(drop3_feat) #0.25
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
#         x = F.relu(self.dense1(x))
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
#         x = F.relu(self.dense2(x))
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

### Functions to Run K-folds 

In [30]:
def run_training(fold, seed_fold, seed_run, display=2):
    seed_everything(seed_run)
    train = folds.copy()
    test_ = test_noctl.copy()
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3)
    
#     loss_fn = nn.BCEWithLogitsLoss()
    loss_val = nn.BCEWithLogitsLoss()
    loss_tr = nn.BCEWithLogitsLoss() #SmoothBCEwLogits(smoothing =1E-3)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), num_targets))
    best_loss = np.inf
    best_auc_loss = 0.0
    best_epoch = 0
    res = []
    
    for epoch in range(EPOCHS):
        
        train_loss, mloss_train = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
#         print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, mloss_valid, y_true, valid_preds = valid_fn(model, loss_val, validloader, DEVICE)
        auc_loss = mean_auc(y_true, valid_preds)
    
        if mloss_valid < best_loss:
            best_loss = mloss_valid
            best_auc_loss = auc_loss
            best_epoch = epoch
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), output_dir / f'ann_model_seedfold{seed_fold}_seedrun{seed_run}_fold__{fold}.pth')
        elif(EARLY_STOP == True):
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
        if scheduler.__class__ !=  torch.optim.lr_scheduler.ReduceLROnPlateau:
            lr = scheduler.get_last_lr()
        else:
            lr = [0.0]
        
        # Save results
        # ------------
        res.append(dict({'epoch':epoch, 'lr':lr[0], 'trn_loss':mloss_train, 'val_loss':mloss_valid, 'auc_loss':auc_loss, 'best_epoch':best_epoch, 'best_loss':best_loss, 'best_auc_loss':best_auc_loss}))
        
        res_df = pd.DataFrame(res)
        res_df.to_csv(output_dir / f'res_seedfold{seed_fold}_seedrun{seed_run}_fold{fold}_.csv')
        
        fig, ax = plt.subplots(nrows=1, ncols=1 )
        min_val = res_df[['trn_loss','val_loss']].min().min()
        ax.plot(res_df['trn_loss'])
        ax.plot(res_df['val_loss'])
        plt.ylim((min_val,0.020))
        plt.title(f"logloss in fold={fold} min={res_df['val_loss'].min()}")
        fig.savefig(output_dir / f'loss_seedfold{seed_fold}_seedrun{seed_run}_fold{fold}_.png')
        plt.close(fig)
        
        fig, ax = plt.subplots(nrows=1, ncols=1 )
        ax.plot(res_df['lr'])
        plt.title(f"lr in fold={fold}")
        fig.savefig(output_dir / f'lr_seedfold{seed_fold}_seedrun{seed_run}_fold{fold}_.png')
        plt.close(fig)
        
        fig, ax = plt.subplots(nrows=1, ncols=1 )
        ax.plot(res_df['auc_loss'])
        plt.title(f"auc_loss in fold={fold} best={res_df['best_auc_loss']}")
        fig.savefig(output_dir / f'auc_loss_seedfold{seed_fold}_seedrun{seed_run}_fold{fold}_.png')
        plt.close(fig)
        if display==2:
            print(f"seed_fold:{seed_fold} seedrun{seed_run} FOLD:{fold} EPOCH:{epoch:2d} lr:{lr[0]:.9f} trn_loss:{mloss_train:.6f} val_loss:{mloss_valid:.6f} auc_loss={auc_loss:.3f} best_epoch:{best_epoch} best_loss:{best_loss:.6f} best_auc_loss={best_auc_loss:.3f}")
    
    if display>=1:
        print(f"BEST!! seed_fold:{seed_fold} seedrun{seed_run} FOLD:{fold} EPOCH:{epoch:2d} lr:{lr[0]:.9f} trn_loss:{mloss_train:.6f} val_loss:{mloss_valid:.6f} auc_loss={auc_loss:.3f} best_epoch:{best_epoch} best_loss:{best_loss:.6f} best_auc_loss={best_auc_loss:.3f}")
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(output_dir / f'ann_model_seedfold{seed_fold}_seedrun{seed_run}_fold__{fold}.pth'))
    model.to(DEVICE)
    
    predictions_tst = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions_tst = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions_tst, best_loss, best_auc_loss


In [31]:
def run_k_fold(NFOLDS, seed_fold, seed_run, display=2):
    global losses_list
    oof_kfold = np.zeros((len(folds), len(target_cols)))
    predictions_kfold = np.zeros((len(test_noctl), len(target_cols)))
    for fold in range(NFOLDS):
        oof_, pred_, val_loss_, best_auc_loss_ = run_training(fold, seed_fold, seed_run, display)
        predictions_kfold += pred_ / NFOLDS
        oof_kfold += oof_
        losses_list.append({'seed_fold':seed_fold, 'seed_run':seed_run, 'fold':fold, 'val_loss':val_loss_, 'auc_loss':best_auc_loss_})
    return oof_kfold, predictions_kfold

In [32]:
feature_cols_ini

['pca_G-0',
 'pca_G-1',
 'pca_G-2',
 'pca_G-3',
 'pca_G-4',
 'pca_G-5',
 'pca_G-6',
 'pca_G-7',
 'pca_G-8',
 'pca_G-9',
 'pca_G-10',
 'pca_G-11',
 'pca_G-12',
 'pca_G-13',
 'pca_G-14',
 'pca_G-15',
 'pca_G-16',
 'pca_G-17',
 'pca_G-18',
 'pca_G-19',
 'pca_G-20',
 'pca_G-21',
 'pca_G-22',
 'pca_G-23',
 'pca_G-24',
 'pca_G-25',
 'pca_G-26',
 'pca_G-27',
 'pca_G-28',
 'pca_C-0',
 'pca_C-1',
 'pca_C-2',
 'pca_C-3',
 'g-0',
 'g-1',
 'g-2',
 'g-3',
 'g-4',
 'g-6',
 'g-8',
 'g-11',
 'g-12',
 'g-13',
 'g-14',
 'g-16',
 'g-17',
 'g-19',
 'g-20',
 'g-21',
 'g-22',
 'g-24',
 'g-26',
 'g-27',
 'g-28',
 'g-29',
 'g-30',
 'g-32',
 'g-33',
 'g-34',
 'g-35',
 'g-36',
 'g-37',
 'g-38',
 'g-39',
 'g-40',
 'g-41',
 'g-43',
 'g-44',
 'g-45',
 'g-46',
 'g-47',
 'g-48',
 'g-49',
 'g-50',
 'g-51',
 'g-52',
 'g-53',
 'g-55',
 'g-56',
 'g-57',
 'g-58',
 'g-59',
 'g-60',
 'g-61',
 'g-62',
 'g-63',
 'g-64',
 'g-65',
 'g-66',
 'g-67',
 'g-68',
 'g-69',
 'g-70',
 'g-71',
 'g-72',
 'g-73',
 'g-75',
 'g-76',
 'g-77'

### Train model

In [33]:
folds_cp = folds.copy()
feature_cols = feature_cols_ini.copy()
NFOLDS = CFG.num_folds
num_features=len(feature_cols)
num_targets=len(target_cols)

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False
hidden_size = 256
res = []
drop1_feat = 0.05
drop2_feat = 0.05
drop3_feat = 0.25

#https://www.kaggle.com/felipebihaiek/torch-continued-from-auxiliary-targets-smoothing
# EPOCHS = 200
# BATCH_SIZE = 512
# LEARNING_RATE = 2e-4
# WEIGHT_DECAY = 2e-7
# EARLY_STOPPING_STEPS = 10
# EARLY_STOP = False

# Averaging on multiple SEEDS and different FOLDS
SEED = np.arange(16)
oof_seed = np.zeros((len(folds), len(target_cols)))
predictions = np.zeros((len(test_noctl), len(target_cols)))
losses_list = []
folds_stds = []
scores_seed = []
for seed in SEED:
    seed_fold = seed // 4
    seed_run = seed % 4
    folds['kfold'] = calc_folds(seed_fold)
    
#     print(folds['kfold'].value_counts())
    print(folds['kfold'].values[:20])
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed_fold, seed_run, display=0)
    oof_seed += oof_ 
    predictions += predictions_
    
    # OBTAIN METRICS
    # --------------
    # Loop score
    folds_cp[target_cols] = oof_
    y_true = folds[target_cols].values 
    y_pred = folds_cp[target_cols].values 
    score_without = log_loss_multi(y_true, y_pred)
    auc_without = mean_auc(y_true, y_pred)
    
    # Score acum
    folds_cp[target_cols] = oof_seed / (seed+1)
    y_true_acum = folds[target_cols].values 
    y_pred_acum = folds_cp[target_cols].values 
    score_without_acum = log_loss_multi(y_true_acum, y_pred_acum)
    auc_without_acum = mean_auc(y_true_acum, y_pred_acum)
    
    losses_df = pd.DataFrame(losses_list)
    folds_loss_mean = losses_df['val_loss'].mean()
    folds_loss_std = losses_df['val_loss'].std()
    folds_auc_mean = losses_df['auc_loss'].mean()
    folds_auc_std = losses_df['auc_loss'].std()
    folds_loss_std_mean = losses_df.groupby(['seed_fold','seed_run'])['val_loss'].std().groupby('seed_fold').mean().mean()
    auc_std_mean = losses_df.groupby(['seed_fold','seed_run'])['auc_loss'].std().groupby('seed_fold').mean().mean()
    
    print(f"SEED={seed} seed_fold={seed_fold} hidden_size={hidden_size} drop1_feat={drop1_feat}\n\
          score_without:{score_without:.6f} auc_without:{auc_without:.3f}\n\
          score_without_acum:{score_without_acum:.6f} ({folds_loss_mean:.6f}, {folds_loss_std:.6f}) error:{folds_loss_std_mean:.6f}\n\
          auc_without_acum:{auc_without_acum:.3f} ({folds_auc_mean:.6f}, {folds_auc_std:.6f}) error:{auc_std_mean:.3f}")
              
    scores_seed.append(dict(seed=seed, seed_fold=seed_fold, hidden_size=hidden_size, drop1_feat=drop1_feat, 
                            score_without=score_without, auc_without=auc_without,
                            score_without_acum=score_without_acum, folds_loss_mean=folds_loss_mean, folds_loss_std=folds_loss_std, folds_loss_std_mean=folds_loss_std_mean,
                            auc_without_acum=auc_without_acum, folds_auc_mean=folds_auc_mean, folds_auc_std=folds_auc_std, auc_std_mean=auc_std_mean))
    
    
    # SUBMISSION
    # ----------------
    test_noctl_cp_acum = test_noctl.copy()
    for col in target_cols:
        test_noctl_cp_acum[col] = 0.0
    test_noctl_cp_acum[target_cols] = predictions / (seed+1)
    
    # Submission acum
    submission = sample_submission.drop(columns=target_cols)\
    .merge(test_noctl_cp_acum[['sig_id']+target_cols], on='sig_id', how='left')\
    .fillna(0.0).reset_index(drop=True)
    # sub.to_csv('submission.csv', index=False)
    name_sub = output_dir / f'submission_seed_fold{seed_fold}__score{score_without_acum:.6f}.csv'
    submission.to_csv(name_sub, index=False)

    # Keep results
    scores_seed_df = pd.DataFrame(scores_seed)
    losses_df.to_csv(output_dir / 'losses_df.csv')
    scores_seed_df.to_csv(output_dir / 'scores_seed_df.csv')
    with open(output_dir / f'ann_dataframes.pkl', 'wb') as file:
        pickle.dump(folds_cp, file)
        pickle.dump(test_noctl_cp_acum, file)
        pickle.dump(y_true_acum, file)
        pickle.dump(y_pred_acum, file)
        pickle.dump(feature_cols, file)
        pickle.dump(target_cols, file)
        pickle.dump(losses_df, file)
        pickle.dump(scores_seed_df, file)

[7 5 3 5 3 4 7 6 9 1 0 2 4 9 4 1 3 4 0 1]
SEED=0 seed_fold=0 hidden_size=256 drop1_feat=0.05
          score_without:0.016005 auc_without:0.779
          score_without_acum:0.016005 (0.016005, 0.000139) error:0.000139
          auc_without_acum:0.779 (0.789130, 0.008520) error:0.009
[7 5 3 5 3 4 7 6 9 1 0 2 4 9 4 1 3 4 0 1]
SEED=1 seed_fold=0 hidden_size=256 drop1_feat=0.05
          score_without:0.016011 auc_without:0.778
          score_without_acum:0.015753 (0.016008, 0.000141) error:0.000145
          auc_without_acum:0.789 (0.787968, 0.006849) error:0.007
[7 5 3 5 3 4 7 6 9 1 0 2 4 9 4 1 3 4 0 1]
SEED=2 seed_fold=0 hidden_size=256 drop1_feat=0.05
          score_without:0.015979 auc_without:0.780
          score_without_acum:0.015654 (0.015998, 0.000132) error:0.000135
          auc_without_acum:0.794 (0.788834, 0.008062) error:0.008
[7 5 3 5 3 4 7 6 9 1 0 2 4 9 4 1 3 4 0 1]
SEED=3 seed_fold=0 hidden_size=256 drop1_feat=0.05
          score_without:0.015979 auc_without:0.777
    

In [34]:
len(feature_cols)

974

In [35]:
pd.read_csv(name_sub).head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib